# Model M/M/1/N

<img src="include/jpTi8JZkULY.jpg">

$$\Omega = \big\{ \omega = \big\{ \omega_{1}, \omega_{2}, ... , \omega_{n}, ... \big\}: \omega_{i} \in \big\{0,1\big\} \big\}$$
$$\omega_{i} = \text{1 - at the time } t_{i} \text{ request arrives}$$
$$\omega_{i} = \text{0 - at the time } t_{i} \text{ request goes away}$$
$$F = 2^{ \Omega } $$
$$P \big( \big\{ \omega_{ i } = 1 \big\} \big) = \frac{ \lambda }{ \lambda + \mu } $$
$$P \big( \big\{ \omega_{ i } = 0 \big\} \big) = \frac{ \mu }{ \lambda + \mu }$$

*Transition rate matrix:*

$$\mathbb{P} = \begin{bmatrix}\quad & \big(0\big) & \big(1\big) & \big(2\big) & \big(3\big) & ... & \big(N-1\big) & \big(N\big) \\ \big(0\big) & 0 & \frac{ \lambda }{ \lambda + \mu } & 0 & 0 & ... & 0 & 0 \\ \big(1\big) & \frac{ \mu }{ \lambda + \mu } & 0 & \frac{ \lambda }{ \lambda + \mu } & 0 & ... & 0 & 0 \\ \big(2\big) & 0 & \frac{ \mu }{ \lambda + \mu } & 0 & \frac{ \lambda }{ \lambda + \mu } & ... & 0 & 0 \\ \big(3\big) & 0 & 0 & \frac{ \mu }{ \lambda + \mu } & 0 & ... & 0 & 0 \\ ... & ... & ... & ... & ... & ... & ... & ... \\ \big(N-1\big) & 0 & 0 & 0 & 0 & ... & 0 & \frac{ \lambda }{ \mu + \lambda } \\ \big(N\big) & 0 & 0 & 0 & 0 & ... & \frac{ \mu }{ \lambda + \mu } & 0 \end{bmatrix}$$

*Get stationary distribution*

$$\Pi_{0} = \frac{1}{ \sum_{i = 0}^{n} \sum_{j = 0}^{i} \big( \frac{ \lambda }{ \mu } \big)^j }$$
$$ \Pi_{i} = \Pi_{i - 1 } + \big( \frac{ \lambda }{ \mu } \big)^{i} \Pi_{0}$$

In [15]:
def get_stationary_distribution(lmbda = 1/5, mu = 1/7, cap = 3):
    suma = 0
    for i in range(cap):
        for j in range(i+1):
            suma += (lmbda / mu) ** i
    pi_0 = 1/ suma
    return pi_0

get_stationary_distribution()

0.1033057851239669

# Model definition

In [ ]:
import time
import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
def get_next_time(param):
    s = 0
    while s == 0:
        # Interval between arrivals averaging (1/param) seconds
        s = random.expovariate(param)
    return s


def MM1Ksimulation(capacity = 1, lmbd:float = 1, mu:float = 1, timeout = 1):
    assert capacity, "Max length of queue should be greater than 0"
    assert lmbd > 0 and mu > 0, "Your model parameters lambda and mu should be greater than 0"
    assert timeout, "Your experiment time should be greater than 0"

    queue, arrival_time, departure_time, num_in_q, waiting_time, wasted_time = [], [], [], [], [], []
    total_customers, lost_customers, total_served, waiting_customers = 0, 0, 0, 0

    # Compute time of next arrival.
    nextArrival = get_next_time(lmbd)

    # TODO(HabKaffee): входной поток - пуассоновский, задержка между приходами и сорвисом - экспонента ???

    # Compute time of next completed service.
    nextService = nextArrival + get_next_time(mu)

    # Simulate the M/M/1 queue.
    timeout_start = time.time() 
    while time.time() < timeout_start + timeout:
        # Next event is an arrival.
        while nextArrival < nextService:
            # Simulate an arrival
            if len(queue) < capacity:
                queue.append(nextArrival)
                arrival_time.append(nextArrival)
                total_customers += 1
                num_in_q.append(len(queue))
                if(len(queue)):
                    waiting_customers += 1      
            else:
                lost_customers  += 1
                total_customers += 1
            nextArrival += get_next_time(lmbd)

        # Next event is a service completion.
        arrival = queue.pop(0)
        total_served += 1
        departure_time.append(nextService)
        waiting_time.append(nextService - arrival)
        num_in_q.append(len(queue))

        # Update the queue.
        if not queue:
            nextService = nextArrival + get_next_time(mu)
        else:
            nextService = nextService + get_next_time(mu)
    num_of_customers = len(arrival_time)
    arrival_time = arrival_time[:len(departure_time)]
    
    for i in range(1, len(departure_time)):
        if ((arrival_time[i]-departure_time[i-1])>0):
            wasted_time.append(arrival_time[i]-departure_time[i-1])
    return      np.mean(waiting_time),              \
                lost_customers/total_customers,     \
                waiting_customers/num_of_customers, \
                np.mean(num_in_q),                  \
                sum(wasted_time)/departure_time[-1]

**Run algo**

In [ ]:
avg_total_time,         \
lost_customers_prob,    \
waiting_customers_prob, \
avg_num_in_q,           \
wasted_time_prob        = MM1Ksimulation(capacity = 250, lmbd = 1/99, mu = 1/5, timeout = 1)
print("Среднее время пребывания в системе",avg_total_time)
print("Вероятность ухода посетителя",lost_customers_prob)
print("Вероятность ожидания", waiting_customers_prob)
print("Среднее количество людей в очереди",avg_num_in_q)
print("Вероятность простоя системы",wasted_time_prob)

# Queuing system characteristics

# Tests

In [ ]:
def print_metrix(parameters, avg_t_t, lost_cust_p, waiting_cust_p, avg_len, wasted_t_p):
    _, axs = plt.subplots(3, 2,figsize=(15,10))

    axs[0][0].plot(parameters, avg_t_t)
    axs[0][0].set_title('Среднее время нахождения в очереди')

    axs[0][1].plot(parameters, lost_cust_p)
    axs[0][1].set_title('Вероятность отказа')

    axs[1][0].plot(parameters, waiting_cust_p)
    axs[1][0].set_title('Вероятность ожидания')

    axs[1][1].plot(parameters, avg_len)
    axs[1][1].set_title('Средняя длина очереди')

    axs[2][0].plot(parameters, wasted_t_p)
    axs[2][0].set_title('Вероятность простоя системы')

**lambda = mu**

In [ ]:
avg_t_t, lost_cust_p, waiting_cust_p, avg_len, wasted_t_p = [], [], [], [], []

parameters = np.arange(0.01, 1, 0.01)
for mu in parameters:
    lmbd = mu
    np.random.seed(int(lmbd*100))
    avg_total_time, lost_customers_prob, waiting_customers_prob, avg_num_in_q, wasted_time_prob = MM1Ksimulation(capacity = 10, lmbd = lmbd, mu = mu, timeout = 0.05)
    avg_t_t.append(avg_total_time)
    lost_cust_p.append(lost_customers_prob)
    waiting_cust_p.append(waiting_customers_prob)
    avg_len.append(avg_num_in_q)
    wasted_t_p.append(wasted_time_prob)

print_metrix(parameters, avg_t_t, lost_cust_p, waiting_cust_p, avg_len, wasted_t_p)

Мемс уменьшением параметров на графе

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

s = []
mu = 1
t = np.arange(0.01, mu/2, 0.01)
#  УМЕНЬШЕНИЕ lambda все нормас
for lmbd in reversed(parameters):
    np.random.seed(int(lmbd*100))
    avg_total_time, lost_customers_prob, waiting_customers_prob, avg_num_in_q, wasted_time_prob = MM1Ksimulation(capacity = 10, lmbd = lmbd, mu = mu, timeout = 0.05)
    s.append(avg_total_time)

fig, ax = plt.subplots()

ax.plot(t, s)
ax.set_xlim(parameters[-1], parameters[0])  # decreasing time
ax.set_xlabel('decreasing time (s)')
ax.set_ylabel('voltage (mV)')
ax.set_title('Should be growing...')
ax.grid(True)

plt.show()

**lambda<mu**

In [ ]:
avg_t_t, lost_cust_p, waiting_cust_p, avg_len, wasted_t_p =[], [], [], [], []
mu = 1
parameters = np.arange(0.01, mu/2, 0.01)
#  УМЕНЬШЕНИЕ lambda все нормас
for lmbd in parameters:
    np.random.seed(int(lmbd*100))
    avg_total_time, lost_customers_prob, waiting_customers_prob, avg_num_in_q, wasted_time_prob = MM1Ksimulation(capacity = 10, lmbd = lmbd, mu = mu, timeout = 0.05)
    avg_t_t.append(avg_total_time)
    lost_cust_p.append(lost_customers_prob)
    waiting_cust_p.append(waiting_customers_prob)
    avg_len.append(avg_num_in_q)
    wasted_t_p.append(wasted_time_prob)

print_metrix(parameters, avg_t_t, lost_cust_p, waiting_cust_p, avg_len, wasted_t_p)

**lambda>mu**

In [ ]:
#metrics
avg_t_t,lost_cust_p,waiting_cust_p,avg_len,wasted_t_p =[],[],[],[],[]
lmbd = 1
parameters = np.arange(0.01, lmbd/2, 0.01)
#  УМЕНЬШЕНИЕ mu все нормас
for mu in parameters:
    np.random.seed(int(mu*100))
    avg_total_time, lost_customers_prob, waiting_customers_prob, avg_num_in_q, wasted_time_prob = MM1Ksimulation(10, lmbd, mu, 0.05)
    avg_t_t.append(avg_total_time)
    lost_cust_p.append(lost_customers_prob)
    waiting_cust_p.append(waiting_customers_prob)
    avg_len.append(avg_num_in_q)
    wasted_t_p.append(wasted_time_prob)

print_metrix(parameters, avg_t_t, lost_cust_p, waiting_cust_p, avg_len, wasted_t_p)

**Зафиксируем lambda=5 и mu=2.5 и проведем эксперименты с различными длинами очереди**

In [ ]:
#metrics
avg_t_t,lost_cust_p,waiting_cust_p,avg_len,wasted_t_p =[],[],[],[],[]
capacity = 100
parameters = np.arange(1, capacity, 1)
for cap in parameters:
    np.random.seed(int(cap*100))
    avg_total_time, lost_customers_prob, waiting_customers_prob, avg_num_in_q, wasted_time_prob = MM1Ksimulation(cap, 0.5, 0.5, 0.05)
    avg_t_t.append(avg_total_time)
    lost_cust_p.append(lost_customers_prob)
    waiting_cust_p.append(waiting_customers_prob)
    avg_len.append(avg_num_in_q)
    wasted_t_p.append(wasted_time_prob)
    
print_metrix(parameters, avg_t_t, lost_cust_p, waiting_cust_p, avg_len, wasted_t_p)

# SHIT

In [ ]:
def print_some_metrics(_capacity, _lambda, _mu, _timeout):
    avg_total_time,         \
    lost_customers_prob,    \
    waiting_customers_prob, \
    avg_num_in_q,           \
    wasted_time_prob        = MM1Ksimulation(capacity = _capacity, lmbd = _lambda, mu = _mu, timeout = _timeout)
    print("Среднее время пребывания в системе",avg_total_time)
    print("Вероятность ухода посетителя",lost_customers_prob)
    print("Вероятность ожидания", waiting_customers_prob)
    print("Среднее количество людей в очереди",avg_num_in_q)
    print("Вероятность простоя системы",wasted_time_prob)



In [ ]:
lambdas = list(np.linspace(0.03, 1, 10))
mus = list(np.linspace(0.03, 1, 10))
capacities = [x for x in range(50, 300, 50)]
timeouts = [x for x in range(1, 20, 3)]
#print(lambdas, mus, capacity, timeouts)
# for capacity in capacities:
#     for timeout in timeouts:
capacity = 50
timeout = 2
for lmbd in lambdas:
    for mu in mus:
        print(f"Lambda = {lmbd}, Mu = {mu}, capacity = {capacity}, timeout = {timeout}\n")
        print_some_metrics(capacity, lmbd, mu, timeout)
        print()
